# Data Science Capstone, Part 1:
### Obtaining and cleaning a dataframe of postal codes, boroughs and neighbourhoods in Toronto

This notebook includes the codes for scraping a wikipedia page containing postal codes and associated boroughs and neighbourhoods in Toronto, turning them into a Pandas dataframe and cleaning it. This is the first part of the [Data Science Capstone project](https://www.coursera.org/learn/applied-data-science-capstone/home/welcome) from IBM's professional specialisation in Data Science on Coursera.

In [1]:
# Import libraries required for this part

import pandas as pd
import numpy as np

Scraping the wikipedia page containing the Toronto postcodes can be easily done with the *read_html* method of Pandas. The method search for $<table>$ tags in the given html page and returns a list of all the found tables converted in Pandas' dataframes on the basis of the internal structure of the tables in the web page (as defined by the $<tr>$ and $<td>$ tags).  

In [34]:
url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # The url of the Wikipedia page containing the postal codes
toronto_postcodes = pd.read_html(url)[0] # The first element in the returned list is the postal codes' table we are interested in 
print("Table succesfully scraped. First 5 rows of it are shown below: ")
toronto_postcodes.head()

Table succesfully scraped. First 5 rows of it are shown below: 


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [35]:
print("Equally, last 5 rows look like this: ")
toronto_postcodes.tail()

Equally, last 5 rows look like this: 


,Postal Code,Borough,Neighbourhood
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,Not assigned


At this point we can start cleaning the dataframe. As a first thing, we want to make sure that each postcode appears just once and that, whether there are duplicated rows in terms of postal code, such rows are aggregated with Neighbourhoods separated by a comma.

In [36]:
# The transform method allows to pass an arbitrary function to aggregate the grouped results. In this case, the join method
# will concatenate the Neighbourhoods having same postcode. After it, duplicated rows thus created can be dropped.

toronto_postcodes['Neighbourhood'] = toronto_postcodes.groupby(['Postal Code'])['Neighbourhood'].transform(lambda x : ', '.join(x))
toronto_postcodes = toronto_postcodes.drop_duplicates() # The above code will create duplicates (if postcodes are repeated). 
                                                        #  This code will drop such duplicates. 

After this transformation, the below code will check if indeed all postcodes appear in the tables just once, by counting their frequencies and assure that no frequency is bigger than 1 occurrence. If there are duplicates, a warning message is printed, else a message confirming the success of this preprocessing step is returned.

In [37]:
exist_duplicate_postcodes = False
for value in toronto_postcodes['Postal Code'].value_counts():
    if value>1:
        exist_duplicate_postcodes = True
if exist_duplicate_postcodes:
    print("Warning: The same postcode has been found more than once! Try to group the dataframe by postcode.")
else:
    print("All Good: each postcode appears just once.")

All Good: each postcode appears just once.


The second preprocessing step concerns the "Not assigned" values present in the table. As a first thing, we substitute them with NaN values from numpy library, as it is easier to work with such values in Pandas.

In [39]:
toronto_postcodes.replace('Not assigned', np.nan, inplace = True)

In [40]:
# Check that the "Not assigned" values were correctly substituted with NaN

toronto_postcodes.head()

,Postal Code,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Then, we make sure that if Neighbourhood has a NaN value, but the Borough from the same row is assigned, then the Neighbourhood is set to be the same name as the Borough.

In [19]:
# By indexing with the .loc method, we can change subsets of the dataframe matching a specific condition (here, neighbourhood
# being NaN) by substituting them with a list of values having same length (here, the value of borough in those same rows).

toronto_postcodes.loc[toronto_postcodes['Neighbourhood']==np.nan,'Neighbourhood'] =  toronto_postcodes[
    toronto_postcodes['Neighbourhood']==np.nan]['Borough'].values

We now check that the operation was succesful by meeting the condition that the length of the dataframe's subset having NaN values for neighbourhood and real values for boroughs is equal to 0. In that case, a message confirming it is printed, else a warning message advises to replace the affected NaN values.

In [41]:
if len(toronto_postcodes[(toronto_postcodes['Neighbourhood']==np.nan) & (toronto_postcodes['Borough']!=np.nan)])>0:
    print("Warning: Some NaN values for Neighbourhood exist, where Borough is not NaN! Try replace the NaN for Neighbourhood\
    with the value of Borough from the same row.")
else:
    print("All Good: You can drop rows containing NaN now.")

All Good: You can drop rows containing NaN now.


Finally, we can drop the rows containing the remaining NaN values, as they are the ones for which we don't have either a borough and a neighbourhood name. As a last point, we reset the index value so that it starts from 0 up to the new length of the dataframe, which is printed in the final cell via the .shape method.

In [42]:
toronto_postcodes.dropna(axis=0, inplace=True)
toronto_postcodes.reset_index(drop=True, inplace=True)
toronto_postcodes.tail()

,Postal Code,Borough,Neighbourhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [43]:
print("New shape of Toronto postcodes dataframe after ignoring entries containing\
NaN values is: {} rows and {} columns.".format(toronto_postcodes.shape[0], toronto_postcodes.shape[1]))

New shape of Toronto postcodes dataframe after ignoring entries containingNaN values is: 103 rows and 3 columns.


In [45]:
# The final dataframe is saved in a csv file with the following code
toronto_postcodes.to_csv("TorontoPost.csv")